## **Part 1: Import Libraries**

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import time


## **Part 2: Load and Explore Data**


In [13]:
df = pd.read_csv("data.csv")
for col in df.select_dtypes(include=['number']).columns:
    df[col] = df[col].fillna(df[col].mean())

X = df.drop("Target", axis=1)
y = df["Target"]


categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['number']).columns


numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## **3.Evaluate Model**

In [14]:

import tracemalloc  # For memory usage tracking

def evaluate_model_with_memory(model, X_train, X_test, y_train, y_test):
    tracemalloc.start()  # Start tracking memory usage
    start_time = time.time()

    model.fit(X_train, y_train)

    end_time = time.time()
    current, peak_memory = tracemalloc.get_traced_memory()  # Memory usage in bytes
    tracemalloc.stop()  # Stop memory tracking

    y_pred = model.predict(X_test)
    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average="weighted"),
        "Recall": recall_score(y_test, y_pred, average="weighted"),
        "F1-Score": f1_score(y_test, y_pred, average="weighted"),
        "Execution Time (s)": end_time - start_time,
        "Memory Used (KB)": peak_memory / 1024  # Convert to KB
    }
    return metrics



## **4.Random Forest**

In [15]:
random_forest = RandomizedSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_distributions={"n_estimators": [50, 100], "max_depth": [None, 10]},
    n_iter=4,
    cv=3,
    random_state=42,
    n_jobs=-1
)

pipeline_rf = Pipeline(steps=[('preprocessor', preprocessor), ('model', random_forest)])
rf_metrics = evaluate_model_with_memory(pipeline_rf, X_train, X_test, y_train, y_test)

print("Random Forest Metrics:\n", rf_metrics)


Random Forest Metrics:
 {'Accuracy': 0.8951428571428571, 'Precision': 0.8991633816467566, 'Recall': 0.8951428571428571, 'F1-Score': 0.8939176611006053, 'Execution Time (s)': 58.67762207984924, 'Memory Used (KB)': 64147.40234375}


## **5.MLP**

In [16]:
mlp_model = RandomizedSearchCV(
    MLPClassifier(random_state=42, max_iter=300),
    param_distributions={"hidden_layer_sizes": [(50,), (100,)], "activation": ["relu", "tanh"], "learning_rate_init": [0.001, 0.01]},
    n_iter=4,
    cv=3,
    random_state=42,
    n_jobs=-1
)

pipeline_mlp = Pipeline(steps=[('preprocessor', preprocessor), ('model', mlp_model)])
mlp_metrics = evaluate_model_with_memory(pipeline_mlp, X_train, X_test, y_train, y_test)

print("MLP Metrics:\n", mlp_metrics)


MLP Metrics:
 {'Accuracy': 0.8681428571428571, 'Precision': 0.8700859439264315, 'Recall': 0.8681428571428571, 'F1-Score': 0.8661660132585184, 'Execution Time (s)': 488.0951340198517, 'Memory Used (KB)': 64218.064453125}


## **6.SVM**

In [ ]:
svm_model = RandomizedSearchCV(
    SVC(probability=True, random_state=42),
    param_distributions={"C": [0.1, 1, 10], "kernel": ["linear", "rbf"]},
    n_iter=4,
    cv=3,
    random_state=42,
    n_jobs=-1
)

pipeline_svm = Pipeline(steps=[('preprocessor', preprocessor), ('model', svm_model)])
svm_metrics = evaluate_model_with_memory(pipeline_svm, X_train, X_test, y_train, y_test)

print("SVM Metrics:\n", svm_metrics)


## **7.Combine results from all models**


In [ ]:
results = {
    "Random Forest": rf_metrics,
    "SVM": svm_metrics,
    "MLP": mlp_metrics
}

results_df = pd.DataFrame(results).T

print("\nComparison of Models:")
print(results_df)


